In [ ]:
## install the pytorch3d package; hit 'shift+enter' keys to run
!conda create --name pytorch3d --yes
!conda activate pytorch3d
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch --yes
!conda install -c conda-forge -c takatosp1 fvcore --yes
!conda install pytorch3d -c pytorch3d --yes
!python3 -m pip install tqdm pandas scipy opencv-python 
!python3 -m pip install --upgrade matplotlib

In [116]:
## download the concerthall optimization code base; hit 'shift+enter' keys to run
!rm -rf /storage/acadia2020_graphcnn
!git clone https://github.com/alexacarlson/acadia2020_graphcnn.git /storage/acadia2020_graphcnn

Cloning into '/storage/acadia2020_graphcnn'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 761 (delta 77), reused 0 (delta 0), pack-reused 630
Receiving objects: 100% (761/761), 6.26 MiB | 11.00 MiB/s, done.
Resolving deltas: 100% (294/294), done.
Checking connectivity... done.
Checking out files: 100% (279/279), done.


In [115]:
## Train the graph convolutional neural network; hit 'shift+enter' keys to run
## NOTES on Training the Graph Convolutional Neural network to map 3D meshes --> aesthetic and functionality parameters:
## To train your own graph convolutional network, you first need to create a folder in paperspace storage 
## at `/storage/concert_dataset` upload your data there. All of your meshes should be `.obj` files and
## should exist in a subdirectory of the dataset: `/storage/acadia2020_graphcnn_dataset/Separated/`. 
## You will need to have a csv file of label/parameters (where each line is in the format `mesh.obj label1 label2 ... label10`) 
## located at and named `/storage/acadia2020_graphcnn_dataset/NamingBookV2.csv`.
## The results (network weights and training loss values) will be saved to `/storage/acadia2020_graphcnn_training_results`.
##
## Note that this function takes in the absolute filepath of a configuration file (CFG_PATH) that sets up the network architecture.
## ## To train on 4 labels:
!cd /storage/acadia2020_graphcnn && bash run_train_graphconv_regression.sh "/storage/acadia2020_graphcnn/config/mesh2aesthetics_train.yml"

RANDOM_SEED: 0
DATA:
  PATH: /storage/acadia2020_graphcnn_dataset
BASE_MODEL:
  CLASSES: 4
  CONV_INIT: normal
  HIDDEN_DIMS: [128, 256, 256, 512]
  INPUT_MESH_FEATS: 3
OPTIM:
  BATCH_SIZE: 10
  CLIP_GRADIENTS: 12.5
  EPOCH: 250
  LR: 0.0002
  MOMENTUM: 0.9
  VAL_BATCH_SIZE: 4
  WEIGHT_DECAY: 0.005
  WORKERS: 8
CHECKPOINT:
  best_acc: 0.0
  best_loss: 1.7976931348623157e+308
  experiment_path: /storage/acadia2020_graphcnn_training_results/exp_09_26_16_15_28_ai_aesthetics_optim
  full_experiment_name: exp_09_26_16_15_28_ai_aesthetics_optim

config_yml          : /storage/acadia2020_graphcnn/config/mesh2aesthetics_train.yml
config_override     : []
Splitting Dataset..
Training Samples: 6
Validation Samples: 2

 ***************** Training *****************
  0%|                                                   | 0/250 [00:00<?, ?it/s]Epoch: 0

	Training Loss: 3.972216526667277
  0%|▏                                          | 1/250 [00:00<02:58,  1.40it/s]Epoch: 1

	Training Loss: 2.8740

In [117]:
## run the optimization framework; hit 'shift+enter' keys to run
## IMPORTANT INFO about running the acoustic optimization script:
## The Command Format for this script: 
## bash run_acousticoptim.sh INPUT_MESH SILHOUETTE_DEFORM_FLAG SILHOUETTE_REF_IMG ACOUSTIC_DEFORM_FLAG WHICH_ACOUSTIC_PARAMS OUTPUT_NAME
## where `INPUT_MESH_FILEPATH` is the absolute filepath of the mesh you would like to deform
##       `TRAINED_GRAPH_FILEPATH` is the filepath to the trained mesh2aesthetic graph convolutional neural network
##       `STYLE_PARAM` is either 'baroque', 'modern', or 'classic' 
##       `SEM_PARAM`  is either 'house' or 'column'
##       `FUNC_PARAM` is an integer number from [1,2,3,4], where 1 is low functionality and 4 is high functionality
##       `AESTH_PARAM` is an integer number from [1,2,3,4], where 1 is poor aesthetics and 5 is good aesthetics 
##       `OUTPUT_FILENAME` is the absolute filepath where you would like the deformed output mesh to be saved.
##       `NUM_ITERS` is the number of iterations to run the optimization.
##       `CFG_FILE` is the absolute path to the configuration file. You will need to change this to the config file path that you used for training the model (see above cell)
##
## Example Command:
#!cd /storage/concerthall && bash run_acousticoptim.sh \
#                            INPUT_MESH_FILEPATH \
#                            TRAINED_GRAPH_FILEPATH \
#                            STYLE_PARAM \
#                            SEM_PARAM \
#                            FUNC_PARAM \
#                            AESTH_PARAM \
#                            OUTPUT_FILENAME \
#                            NUM_ITERS \
#                            CFG_FILE
#
!cd /storage/acadia2020_graphcnn && bash run_param_optim.sh \
                            "/storage/3Dmodels/bench.obj" \
                            "/storage/acadia2020_graphcnn_training_results/exp_09_26_14_01_06_ai_aesthetics_optim/model_best_loss.pkl" \
                            "modern" \
                            "house" \
                            4 \
                            5 \
                            "test_sil_deform.obj" \
                            250 \
                            "/storage/acadia2020_graphcnn/config/mesh2aesthetics_train.yml"


Saving optim results to /storage/results_bench_optim_stylemodern_semantichouse_func4_aesth5
[tensor([0], device='cuda:0'), tensor([0], device='cuda:0'), tensor([3], device='cuda:0'), tensor([4], device='cuda:0')]

 ***************** Deforming *****************
torch.Size([1, 2]) torch.Size([1]) tensor([0], device='cuda:0')
torch.Size([1, 2]) torch.Size([1]) tensor([0], device='cuda:0')
torch.Size([1, 4]) torch.Size([1]) tensor([3], device='cuda:0')
torch.Size([1, 5]) torch.Size([1]) tensor([4], device='cuda:0')
Iteration: 0 Loss: 918.22626
torch.Size([1, 2]) torch.Size([1]) tensor([0], device='cuda:0')
torch.Size([1, 2]) torch.Size([1]) tensor([0], device='cuda:0')
torch.Size([1, 4]) torch.Size([1]) tensor([3], device='cuda:0')
torch.Size([1, 5]) torch.Size([1]) tensor([4], device='cuda:0')
torch.Size([1, 2]) torch.Size([1]) tensor([0], device='cuda:0')
torch.Size([1, 2]) torch.Size([1]) tensor([0], device='cuda:0')
torch.Size([1, 4]) torch.Size([1]) tensor([3], device='cuda:0')
torch.